# This notebook is a tutorial for GoogleMap image extraction
* If you want to know more about **map tiles**, go to https://developers.google.com/maps/documentation/javascript/coordinates
  
* If you want to know more about **map types**, go to https://developers.google.com/maps/documentation/javascript/maptypes
  
* About the **zoom** parameter :
Offering a map of the entire Earth as a single image would either require an immense map, or a small map with very low resolution. As a result, map images within Google Maps and the Maps JavaScript API are broken up into map "tiles" and "zoom levels." At low zoom levels, a small set of map tiles covers a wide area; at higher zoom levels, the tiles are of higher resolution and cover a smaller area. The following list shows the approximate level of detail you can expect to see at each zoom level:
    * 1: World
    * 5: Landmass/continent
    * 10: City
    * 15: Streets
    * 20: Buildings
<img src="tuto_images/10.PNG" width=500>

In [ ]:
class GoogleMapDownloader:
    """A class which generates high resolution google maps images given a longitude, latitude and zoom level"""
    def __init__(self, lng, lat, **kwargs):
        """
          GoogleMapDownloader Constructor
          Args:
            lat:    The latitude of the location required
            lng:    The longitude of the location required
            zoom:   The zoom level of the location required, ranges from 0 - 23, defaults to 16
        """
        self._lat   = lat
        self._lng   = lng
        self._zoom  = kwargs.get('zoom', 16)
        self._scale = 156543.03392 * math.cos(lat * math.pi / 180) / math.pow(2, self._zoom)

    def fromLngLatToPoint(self):
        """Generates an X,Y tile coordinate based on the latitude, longitude and zoom level
        Returns:    An X,Y tile coordinate
        """
        tile_size = 256
        # Use a left shift to get the power of 2 i.e. a zoom level of 2 will have 2^2 = 4 tiles
        numTiles = 1 << self._zoom

        # Find the x_point given the longitude
        point_x = (tile_size/ 2 + self._lng * tile_size / 360.0) * numTiles // tile_size
        # Convert the latitude to radians and take the sine
        sin_y = math.sin(self._lat * (math.pi / 180.0))
        # Calulate the y coorindate
        point_y = ((tile_size / 2) + 0.5 * math.log((1+sin_y)/(1-sin_y)) * -(tile_size / (2 * math.pi))) * numTiles // tile_size
        return int(point_x), int(point_y)


    def fromPointToLngLat(self,x,y):
        lat = (2 * math.atan(math.exp((y - 128) / -(256 / (2 * math.pi)))) - math.pi / 2)/ (math.pi / 180)
        lon = (x - 128) / (256 / 360)
        return lon,lat

    def getTileBounds(self,tile_width,tile_height):
        x,y = self.fromLngLatToPoint()
        s = 256/(1 << self._zoom)
        nw=(x*s,y*s)
        se=(x*s+tile_width*s,y*s+tile_height*s)
        return {'nw':self.fromPointToLngLat(*nw),'se':self.fromPointToLngLat(*se)}

    def generateImage(self,map_type, **kwargs):
        """Generates an image by stitching a number of google map tiles together.
        Arguments:
          map_type:    The type of the map, possible values are :
                              h = roads only
                              m = standard roadmap
                              p = terrain
                              r = somehow altered roadmap
                              s = satellite only
                              t = terrain only
                              y = hybrid
          start_x:      The top-left x-tile coordinate
          start_y:      The top-left y-tile coordinate
          tile_width:   The number of tiles wide the image should be - defaults to 1
          tile_height:  The number of tiles high the image should be - defaults to 1
        Returns:
            A high-resolution Goole Map image.
        """
        from urllib.request import urlretrieve

        start_x = kwargs.get('start_x', None)
        start_y = kwargs.get('start_y', None)
        tile_width  = kwargs.get('tile_width', 1)
        tile_height = kwargs.get('tile_height', 1)

        # Check that we have x and y tile coordinates
        if start_x == None or start_y == None :
            start_x, start_y = self.fromLngLatToPoint()

        # Determine the size of the image
        width, height = 256 * tile_width, 256 * tile_height

        #Create a new image of the size require
        map_img = Image.new('RGB', (width,height))

        for x in range(0, tile_width):
            for y in range(0, tile_height) :
                url = 'https://mt0.google.com/vt/lyrs='+map_type+'&x='+str(start_x+x)+'&y='+str(start_y+y)+'&z='+str(self._zoom)
                current_tile = str(x)+'-'+str(y)
                urlretrieve(url, current_tile)

                im = Image.open(current_tile)
                map_img.paste(im, (x*256, y*256))

                os.remove(current_tile)
        coords = self.getTileBounds(tile_width,tile_height)
        return map_img,coords

In [ ]:
folder = ''

#Create a new instance of GoogleMap Downloader
gmd = GoogleMapDownloader(lon, lat) #you can specify zoom=...

try: 
    # Get the high resolution image
    img,coords = gmd.generateImage(map_type) #you can specify start_x=.., start_y=.., tile_width=.., tile_height=..
except IOError:
    print("Could not generate the image - try adjusting the zoom level and checking your coordinates")
else:
    #Save the image to disk
    slon = str(round(lon,5)).replace('.',',')
    slat = str(round(lat,5)).replace('.',',')
    filename = "image_"+slon+'_'+slat
    if not(os.path.isfile('data/'+map_type+'data/'+filename+'.png')):
        img.save(folder+'/'+filename+'.png')
    print('image size : ',img.size)
    print('scale : ',gmd._scale)
    print('corners are : ',coords)